In [1]:
import pandas as pd
import datetime as dt
import time as tm

data = pd.read_csv('data.tsv', sep='\t')
data.head()

,id,date,user_id,duration,medium,source,cost,order_id,amount_paid
0,40443,05.10.2016 23:18,1010,0.000926,seo,google,0.00,6243,20.2
1,35044,09.10.2016 21:40,1036,0.006493,sem,yandex,0.07,6145,15.6
2,40177,05.10.2016 3:23,1041,0.003380,email,promo,0.00,6128,13.2
3,39401,05.10.2016 23:19,1041,0.000463,sem,yandex,0.03,6697,9.8
4,41545,01.10.2016 4:57,1042,0.006493,sem,google,0.06,4510,14.8


In [2]:
# функция для перевода строки в дату
def convert_to_datetime(row):
    return dt.datetime.strptime(row['date'], '%d.%m.%Y %H:%M')

In [3]:
# применяем функцию и создаем новый столбец datetime
data['datetime'] = data.apply(convert_to_datetime, axis=1)
data.head()

,id,date,user_id,duration,medium,source,cost,order_id,amount_paid,datetime
0,40443,05.10.2016 23:18,1010,0.000926,seo,google,0.00,6243,20.2,2016-10-05 23:18:00
1,35044,09.10.2016 21:40,1036,0.006493,sem,yandex,0.07,6145,15.6,2016-10-09 21:40:00
2,40177,05.10.2016 3:23,1041,0.003380,email,promo,0.00,6128,13.2,2016-10-05 03:23:00
3,39401,05.10.2016 23:19,1041,0.000463,sem,yandex,0.03,6697,9.8,2016-10-05 23:19:00
4,41545,01.10.2016 4:57,1042,0.006493,sem,google,0.06,4510,14.8,2016-10-01 04:57:00


In [4]:
# переводим время в формат unixtime
def make_unix_time(row):
    return tm.mktime(row['datetime'].timetuple())

In [5]:
# применяем функцию и создаем новый столбец unixtime
data['unixtime'] = data.apply(make_unix_time, axis=1)
data.head()

,id,date,user_id,duration,medium,source,cost,order_id,amount_paid,datetime,unixtime
0,40443,05.10.2016 23:18,1010,0.000926,seo,google,0.00,6243,20.2,2016-10-05 23:18:00,1.475699e+09
1,35044,09.10.2016 21:40,1036,0.006493,sem,yandex,0.07,6145,15.6,2016-10-09 21:40:00,1.476038e+09
2,40177,05.10.2016 3:23,1041,0.003380,email,promo,0.00,6128,13.2,2016-10-05 03:23:00,1.475627e+09
3,39401,05.10.2016 23:19,1041,0.000463,sem,yandex,0.03,6697,9.8,2016-10-05 23:19:00,1.475699e+09
4,41545,01.10.2016 4:57,1042,0.006493,sem,google,0.06,4510,14.8,2016-10-01 04:57:00,1.475287e+09


In [6]:
# группируем датафрейм data по столбцу user_id, посчитав для столбца unixtime максимальное и минимальное значение для каждого пользователя
data_grouped = data.groupby('user_id').agg([min, max])['unixtime']
data_grouped.reset_index()

,user_id,min,max
0,1010,1.475699e+09,1.475699e+09
1,1036,1.476038e+09,1.476038e+09
2,1041,1.475627e+09,1.475699e+09
3,1042,1.475287e+09,1.475934e+09
4,1047,1.475318e+09,1.475693e+09
...,...,...,...
119,1393,1.475721e+09,1.475721e+09
120,1394,1.475614e+09,1.475614e+09
121,1397,1.475633e+09,1.475633e+09
122,1398,1.475697e+09,1.475697e+09


In [7]:
# считаем столбец diff с разностью максимального и минимального значения столбца unixtime
data_grouped['diff'] = data_grouped['max'] - data_grouped['min']
data_grouped.reset_index()

# исключем из расчета пользователей, у которых разница diff равна 0.
final_data = data_grouped[data_grouped['diff'] > 0]

# считаем среднее значение столбца diff после фильтрации в секундах, переводим его в дни и округляем
round((final_data['diff'].mean() / 60/60/24), 1)

3.5